In [ ]:
import pandas as pd
import numpy as np
import joblib
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2') 
# model_classification = joblib.load("../models/log_classifier.joblib")
import requests

deployed_model_name = "logclassifier"
infer_endpoint = "https://logclassifier.log-analyzer.svc.cluster.local:8008"
infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"

def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "float_input",
                "shape": [1, 384],
                "datatype": "FP32",
                "data": data.tolist()
            }
        ]
    }

    response = requests.post(infer_url, json=json_data)
    response_dict = response.json()
    return response_dict['outputs'][0]['data']

test_log = "2021-08-06 15:32:00,000 INFO  [main] com.example.Main: This is an example log message"
test_embeddings = model.encode([test_log])
result = rest_request(test_embeddings.astype(np.float32))
print(result)

In [5]:


test_df = pd.read_csv('myapp_logs-test.nogit.csv')
test_df['classification'] = ''
for index, row in test_df.iterrows():
    test_log = row['message']
    test_embeddings = model.encode([test_log])
    # test_probabilities = model_classification.predict_proba(test_embeddings)[0]
    test_probabilities = predit_proba_remote(test_embeddings)[0]
    test_predicted_label = 'Unclassified'
    if max(test_probabilities) < 0.5:
        print(test_log, "--> Unclassified -->", test_probabilities)
    else:
        # test_predicted_label = model_classification.predict(test_embeddings)[0]
        test_predicted_label = predit_remote(test_embeddings)[0]
        print(test_log, "->", test_predicted_label, "-->", test_probabilities)
    test_df.at[index, 'classification'] = test_predicted_label

test_df.to_csv('myapp_logs-test_with_classification.nogit.csv', index=False)

TypeError: Object of type ndarray is not JSON serializable